<h1 align='center'> Column Transformer </h1>

In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('covid_toy.csv')

In [3]:
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [5]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [6]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [7]:
df.isnull().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

### train test split

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X= df.drop('has_covid', axis=1)
y=df['has_covid']

In [10]:
X

,age,gender,fever,cough,city
0,60,Male,103.0,Mild,Kolkata
1,27,Male,100.0,Mild,Delhi
2,42,Male,101.0,Mild,Delhi
3,31,Female,98.0,Mild,Kolkata
4,65,Female,101.0,Mild,Mumbai
...,...,...,...,...,...
95,12,Female,104.0,Mild,Bangalore
96,51,Female,101.0,Strong,Kolkata
97,20,Female,101.0,Mild,Bangalore
98,5,Female,98.0,Strong,Mumbai


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

### 1. Aam Zindagi  

- When we don't know how to use column transformer we use tidious way to do encoding

### Simple Imputer

-  For __Fever__ numerical column there are some missing values
- To handle it we use sklearn __SimpleImputer__

In [12]:
from sklearn.impute import SimpleImputer

In [13]:
si = SimpleImputer()

In [14]:
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.fit_transform(X_test[['fever']])

In [39]:
X_train_fever.shape

(80, 1)

### Ordinal Encoding

In [17]:
from sklearn.preprocessing import OrdinalEncoder

In [18]:
X_train['cough'].value_counts()

Mild      48
Strong    32
Name: cough, dtype: int64

In [19]:
oe = OrdinalEncoder(categories=[['Mild','Strong']])
X_train_cough = oe.fit(X_train[['cough']])

# X_train transform
X_train_cough = oe.transform(X_train[['cough']])
X_test_cough = oe.transform(X_test[['cough']])

In [38]:
X_train_cough.shape

(80, 1)

### One Hot Encoding --> Gender, City 

In [21]:
from sklearn.preprocessing import OneHotEncoder

In [27]:
ohe = OneHotEncoder(drop='first', sparse=False) # Gender: we will get 1 Column
                                                # city: 3column

In [28]:
X_train_gender_city = ohe.fit_transform(X_train[['gender','city']])
X_test_gender_city = ohe.fit_transform(X_test[['gender','city']])

In [34]:
X_train_gender_city.shape

(80, 4)

- Now we will concatinate all the arrays horizontally
- Firstly,
- Extract Age from X_train as it is not encoded

In [30]:
X_train_age=X_train.drop(columns=['gender','fever','cough','city']).values

In [31]:
X_test_age = X_test.drop(columns=['gender','fever','cough','city']).values

In [33]:
X_train_age.shape

(80, 1)

In [35]:
# Concatinate  Train Data
X_train_transformed = np.concatenate((X_train_age,X_train_fever, X_train_gender_city, X_train_cough), axis=1)

#Test data
X_test_transformed = np.concatenate((X_test_age,X_test_fever, X_test_gender_city, X_test_cough), axis=1)

In [37]:
X_train_transformed.shape

(80, 7)

### Mentos Zindagi

In [40]:
from sklearn.compose import ColumnTransformer

In [43]:
transformer = ColumnTransformer(transformers=[
    ('tn1',SimpleImputer(),['fever']),
    ('tn2', OrdinalEncoder(categories=[['Mild', 'Strong']]),['cough']),
    ('tn3',OneHotEncoder(drop='first',sparse=False),['gender','city'])
], remainder= 'passthrough')

In [44]:
transformer.fit_transform(X_train)
transformer.fit_transform(X_test)


array([[100.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  19.        ],
       [104.        ,   0.        ,   1.        ,   0.        ,
          0.        ,   0.        ,  25.        ],
       [101.        ,   0.        ,   1.        ,   1.        ,
          0.        ,   0.        ,  42.        ],
       [101.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   1.        ,  81.        ],
       [102.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,   5.        ],
       [100.        ,   0.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  27.        ],
       [103.        ,   0.        ,   0.        ,   0.        ,
          1.        ,   0.        ,  69.        ],
       [ 98.        ,   1.        ,   1.        ,   0.        ,
          1.        ,   0.        ,  34.        ],
       [ 99.        ,   0.        ,   0.        ,   0.        ,
          0.    